# Evaluation Algorithm 

# Data Prep

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = 'add_test_data_file'  # Update with the actual file path
df = pd.read_csv(file_path, delimiter=',', header=None, names=["uri", "pref_label", "altlabel_nl"])
sampled_df = df.sample(n=10, random_state=1)
df = sampled_df
# Initialize a list to store extra labels
labels = []

# Function to process the labels in the third column
def process_cell(cell):
    if pd.isna(cell) or cell.strip() == '':
        return cell  # Return as is if empty or NaN
    cell_labels = [label.strip() for label in cell.split(', ')]  # Split labels using a comma as delimiter
    labels.extend(cell_labels)  # Add all labels to the labels list
    return cell
# Apply the function to the 'altlabel_nl' column
labels = df['altlabel_nl'].apply(process_cell)


print("\nTotal extra labels collected:", len(labels))
print(len(df))


# NLLB - pre-trained and finetuned


**Language codes NLLB**
* eng_Latn: English
* deu_Latn: German
* fra_Latn: French
* ita_Latn: Italian
* nld_Latn: Dutch
* spa_Latn: Spanish
* zho_Hans: Chinese (Simplified)

In [ ]:
import torch
import sentencepiece
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer

#Load the pre-trained model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"  # Multiple to English translation model

tokenizer = NllbTokenizer.from_pretrained(model_name,
                                          src_lang="nld_Latn",
                                          tgt_lang="eng_Latn"
                                          )
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map='auto',
    use_cache=False
    )

NLLB - trained

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer

# Define the directory and model save path
directory = 'url'
#file_name = 'nllb_ft_en_fr.h5' #nllb_ft_src_tgt

#Load the model and tokenizer
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(directory)
loaded_tokenizer = NllbTokenizer.from_pretrained(directory)

#model = loaded_model
#tokenizer = loaded_tokenizer
#Example usage to verify
print("Model and tokenizer saved and loaded successfully.")

Model and tokenizer saved and loaded successfully.


In [ ]:
def translate_nllb(label):
  input_ids = tokenizer.encode(label, return_tensors="pt").to(model.device)

  # Target language code
  translated_ids = model.generate(input_ids, forced_bos_token_id=tokenizer.encode("eng_Latn")[1], max_length=50, num_beams=4, early_stopping=True)
  translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

  return translated_text

In [ ]:
translated_labels = [translate_nllb(label) for label in labels]

# DEEPL API

In [ ]:
import pandas as pd
import deepl

#api key = 6826807b-21a7-48bb-b481-a89caa5967fa:fx
api_key = '6826807b-21a7-48bb-b481-a89caa5967fa:fx'

translator = deepl.Translator(api_key)

def translate_text(text, target_lang='en-us'):
    result = translator.translate_text(text, target_lang=target_lang)
    return result.text

In [ ]:
translated_labels = [translate_text(label) for label in labels]

# Opus model

In [ ]:
import torch
import sentencepiece
from transformers import MarianTokenizer, MarianMTModel

# Load the pre-trained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-mul-en"  # Multiple to English translation model

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [ ]:
def translate(label):
  input_ids = tokenizer.encode(label, return_tensors="pt")
  translated_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
  translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

  return translated_text

translated_labels = [translate(label) for label in labels]
print(translated_labels)

# BERT

Trained BERT

In [ ]:
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer, BertTokenizer, TFAutoModel
import os

directory = 'url'

try:
    # Load the tokenizer
    loaded_tokenizer = BertTokenizer.from_pretrained(directory)

    # Load the model
    loaded_model = TFAutoModel.from_pretrained(directory)

    print("Model and tokenizer loaded successfully!")
except Exception as e:
    print("Error loading model:", e)

# MatSciBERT and BioMedBERT

In [ ]:
from transformers import AutoModelForMaskedLM, BertTokenizer,TFAutoModel, AutoTokenizer, AutoModel
import torch

def load_matscibert():
    """
    Load the pretrained MatSciBERT model and tokenizer

    Returns:
        tuple: (tokenizer, model) pair for MatSciBERT
    """
    model_name = "m3rg-iitd/matscibert"

    # Load tokenizer and model
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = TFAutoModel.from_pretrained(model_name)

    return tokenizer, model

def load_biomedbert():
    """
    Load the pretrained BioMedBERT model and tokenizer

    Returns:
        tuple: (tokenizer, model) pair for BioMedBERT
    """
    model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    return tokenizer, model

In [ ]:
matsci_tokenizer, matsci_model = load_matscibert()
biomed_tokenizer, biomed_model = load_biomedbert()

# Get embeddings from different models

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFAutoModel
from scipy.spatial.distance import cosine
#tf.experimental.numpy.experimental_enable_numpy_behavior()

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFAutoModel.from_pretrained('bert-base-multilingual-cased')

# Load trained BERT

tokenizer = loaded_tokenizer # matsci_tokenizer
model = loaded_model #matsci_model

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=512)
    outputs = model(inputs)
    return tf.reduce_mean(outputs.last_hidden_state, axis=1).numpy().squeeze()


def get_embedding_biomed(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    return embeddings.mean(dim=1).detach().numpy().squeeze()


def get_embedding_matsci(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)

    return tf.reduce_mean(embeddings, axis=1).numpy().squeeze()

def calculate_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1, embedding2)

def max_similarity(pref_label_embeddings, alt_label_embeddings, pref_labels, alt_labels):
    results = []

    for i, pref_label_embedding in enumerate(pref_label_embeddings):
        sim_list = []
        for j, alt_label_embedding in enumerate(alt_label_embeddings):
            similarity_ratio = calculate_similarity(pref_label_embedding, alt_label_embedding)
            sim_list.append((alt_labels[j], similarity_ratio))

        max_alt_label, max_sim_ratio = max(sim_list, key=lambda x: x[1])
        results.append({
            'pref_label': pref_labels[i],
            'alt_label': max_alt_label,
            'max_sim_ratio': max_sim_ratio
        })

        # Print the remaining similarities for record
        print(f"Pref Label: {pref_labels[i]}")
        for alt_label, sim_ratio in sim_list:
            print(f"  Alt Label: {alt_label}, Similarity: {sim_ratio}")

    sim_df = pd.DataFrame(results)
    return sim_df

# Assuming you have precomputed embeddings and labels
pref_labels = df['pref_label'].tolist()
alt_labels = df['altlabel_nl'].tolist()

pref_label_embeddings = [get_embedding(label) for label in pref_labels]
alt_label_embeddings = [get_embedding(label) for label in alt_labels]

sim_df = max_similarity(pref_label_embeddings, alt_label_embeddings, pref_labels, alt_labels)
#sim_df.head(10)


# Entity Linking Logic

BERT

In [ ]:
def link_entities_to_dataframe(sim_df, df):
    threshold_sim = 0
    linked_entities = []
    match_data = []

    for index, row in sim_df.iterrows():
        pref_label = row['pref_label']
        max_sim_ratio = row['max_sim_ratio']
        alt_label = row['alt_label']

        if max_sim_ratio > 
        
        _sim:
            # Find the corresponding row in the original df
            original_row = df[df['pref_label'] == pref_label].iloc[0]
            uri = original_row['uri']

            # Link the entity
            linked_entities.append((uri, pref_label, max_sim_ratio))

            match_data.append({
                'uri': uri,
                'pref_label': pref_label,
                'max_similarity': max_sim_ratio,
                'alt_label': alt_label
            })
    # Print the linked entities
    for entity in linked_entities:
        print(f"Pref: {entity[1]}, Linked Entity: {entity[0]}, Similarity: {entity[2]}")

    match_df_bert = pd.DataFrame(match_data)

    return match_df_bert

In [ ]:
match_df_bert = link_entities_to_dataframe(sim_df, df)
match_df_bert.head(30)

String based

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

user_queries = translated_labels

# Function to calculate string similarity
def calculate_similarity(query, label):
    return fuzz.ratio(query, label)


In [ ]:
def max_similarity(query_list):
    pref_label_list = df['pref_label'].tolist()
    results = []

    for i in pref_label_list:
        sim_list = []
        for user_query in query_list:
            similarity_ratio = calculate_similarity(user_query, i)
            sim_list.append((user_query, similarity_ratio))

        max_query, max_sim_ratio = max(sim_list, key=lambda x: x[1])
        results.append({'pref_label': i, 'query': max_query, 'max_sim_ratio': max_sim_ratio})

    sim_df = pd.DataFrame(results)
    return sim_df

sim_df = max_similarity(user_queries)
sim_df.head(30)

,pref_label,query,max_sim_ratio
0,Benzylpenicillin,benzylpenicillin,94
1,Methylamine,"Methylamine, mono-methylamine",55
2,Vanadium(III) oxide,vanadium(III)oxide,92
3,CuCo,Cuco,75
4,Aluminum phosphide,lithium chloride,59
5,Jamin,Jamin,100
6,Yohimbin,Jamin,46
7,Barium hydroxide,barium hydroxide,94
8,PrCd,PRCD,50
9,Ecdyson,Ecdysone,93


In [ ]:
# Function to link entities with maximum similarity
def link_entities_to_dataframe(translated_labels, df):
    user_queries = translated_labels

    # # Get the similarity DataFrame
    # sim_df = max_similarity(user_queries)
    threshold_sim = 50
    linked_entities = []
    match_data = []

    for index, row in sim_df.iterrows():
        pref_label = row['pref_label']
        max_sim_ratio = row['max_sim_ratio']
        query = row['query']

        if max_sim_ratio > threshold_sim: #max_sim_ratio > threshold_sim
            # Find the corresponding row in the original df
            original_row = df[df['pref_label'] == pref_label].iloc[0]
            uri = original_row['uri']

            # Link the entity
            linked_entities.append((uri, pref_label, max_sim_ratio))

            match_data.append({
                'uri': uri,
                'pref_label': pref_label,
                'Max_similarity': max_sim_ratio,
                'translated_label': query
            })

    #Print the linked entities
    #for entity in linked_entities:
      #print(f"Pref: {entity[1]}, Linked Entity: {entity[0]}, Max_Similarity: {entity[2]}")

    match_df = pd.DataFrame(match_data)

    return match_df

match_df = link_entities_to_dataframe(translated_labels, df)

# Evaluation Logic

String Based Approach

In [ ]:
def find_most_similar_uri(label, df):
    uri_list = []
    distance_list = []

    # Iterate through the DataFrame
    for i, row in df.iterrows():
        # Calculate the similarity ratio between the label and pref_label
        ratio = fuzz.ratio(row.pref_label, label)

        # Store URI and similarity ratio in lists
        uri_list.append(row.uri)
        distance_list.append(ratio)

    # Find the index of the maximum similarity ratio
    max_index = distance_list.index(max(distance_list))

    # Return the URI with the highest similarity ratio
    most_similar_uri = uri_list[max_index]

    return most_similar_uri

In [ ]:
match_df["predicted_uri"] = match_df.translated_label.apply(lambda x: find_most_similar_uri(x, match_df))
match_df["uri_match"] = match_df.uri == match_df.predicted_uri

In [ ]:
print(f"The Entity Linking metric is {match_df.uri_match.sum()/len(match_df)*100} %")

BERT Approach

In [ ]:
def find_most_similar_uri_bert(label, df):
    label_embedding = get_embedding(label)
    uri_list = []
    similarity_list = []

    # Iterate through the DataFrame
    for i, row in df.iterrows():
        # Get the embedding for the pref_label
        pref_label_embedding = get_embedding(row.pref_label)

        # Calculate the similarity ratio between the embeddings
        similarity_ratio = calculate_similarity(label_embedding, pref_label_embedding)

        # Store URI and similarity ratio in lists
        uri_list.append(row.uri)
        similarity_list.append(similarity_ratio)

    # Find the index of the maximum similarity ratio
    max_index = similarity_list.index(max(similarity_list))

    # Return the URI with the highest similarity ratio
    most_similar_uri = uri_list[max_index]

    return most_similar_uri

In [ ]:
match_df_bert["predicted_uri"] = match_df_bert.alt_label.apply(lambda x: find_most_similar_uri_bert(x, match_df_bert))
match_df_bert["uri_match"] = match_df_bert.uri == match_df_bert.predicted_uri

In [ ]:
print(f"The Entity Linking metric is {match_df_bert.uri_match.sum()/len(match_df_bert)*100} %")

#BLUE and METEOR


In [ ]:
import nltk

# Download the wordnet resource
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score


# Function to calculate BLEU and METEOR scores
def calculate_scores(reference, candidate):
    # Normalize case
    reference = reference.lower()
    candidate = candidate.lower()

    # Tokenize the reference and candidate sentences
    reference_tokens = [reference.split()]
    candidate_tokens = candidate.split()

    # Calculate BLEU score
    smoothing_function = SmoothingFunction().method1
    bleu = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothing_function)

    # Calculate METEOR score
    meteor = meteor_score(reference_tokens, candidate_tokens)

    return bleu, meteor

# Assuming df is your DataFrame with 'pref_labels' and 'translated_labels' columns
# Initialize lists to store the scores
bleu_scores = []
meteor_scores = []

for index, row in match_df.iterrows():
    reference = row['pref_label']
    candidate = row['translated_label']
    bleu, meteor = calculate_scores(reference, candidate)
    bleu_scores.append(bleu)
    meteor_scores.append(meteor)
    print(f"Row {index}: BLEU score = {bleu}, METEOR score = {meteor}")# Add the scores to the DataFrame



In [ ]:
match_df['bleu_score'] = bleu_scores
match_df['meteor_score'] = meteor_scores

,uri,pref_label,Max_similarity,translated_label,predicted_uri,uri_match,bleu_score,meteor_score
0,http://km.springer.com/smat-substances/VINYGTS...,Benzylpenicillin,94,benzylpenicillin,http://km.springer.com/smat-substances/VINYGTS...,True,0.177828,0.5
1,http://km.springer.com/smat-substances/TCGSWOL...,Methylamine,55,"Methylamine, mono-methylamine",http://km.springer.com/smat-substances/TCGSWOL...,True,0.000000,0.0
2,http://km.springer.com/smat-substances/XZHRKUV...,Vanadium(III) oxide,92,vanadium(III)oxide,http://km.springer.com/smat-substances/XZHRKUV...,True,0.000000,0.0
3,http://km.springer.com/smat-substances/DPTZTCB...,CuCo,75,Cuco,http://km.springer.com/smat-substances/DPTZTCB...,True,0.177828,0.5
4,http://km.springer.com/smat-substances/DGLXULI...,Aluminum phosphide,59,lithium chloride,http://km.springer.com/smat-substances/YINGDPR...,False,0.000000,0.0


In [ ]:
import pandas as pd

# Calculate the average BLEU score
average_bleu_score = match_df['bleu_score'].mean()

# Calculate the average METEOR score
average_meteor_score = match_df['meteor_score'].mean()

# Print the average scores
print(f"Average BLEU score: {average_bleu_score}")
print(f"Average METEOR score: {average_meteor_score}")

Average BLEU score: 0.1141993778028801
Average METEOR score: 0.3277027027027027
